written for https://stackoverflow.com/q/49939762/425458

In [14]:
x = np.array([3,  2,  3,  3,  1,  3,  3,  2,  4,  3,  4,  3,  2])
time = np.array([0,  2,  4,  7, 10, 11, 12, 17, 21, 25, 29, 30, 40])

def count(x, time, epoch=10):
    ix = time.searchsorted(np.arange(0, time.max(), epoch))
    counts = [np.sum(x[ixb:ixe+1]) for ixb,ixe in zip(ix[:-1], ix[1:])]
    if (time[-1] - time[ix[-1]]) >= epoch:
        return counts + [np.sum(x[ix[-1]:])]
    else:
        return counts
    
count(x,time)

[12, 13, 14, 5]

In [43]:
signal = np.array([3,  2,  3,  3,  1,  3,  3,  2,  4,  3,  4,  3,  2])
time = np.array([0,  2,  4,  7, 10, 11, 12, 17, 21, 25, 29, 30, 40])

from itertools import chain,repeat,accumulate

In [26]:
    %%timeit
    idx = np.fromiter(iter(accumulate(chain((0,), repeat(10)), lambda now, delta: time.searchsorted(time[now] + delta)).__next__, len(time)), int)
    np.add.reduceat(signal[:idx[-1]], idx[:-1]) + signal[idx[1:]]

9.63 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [37]:
# time = np.array([ 2,  2,  4,  7, 10, 11, 12, 17, 21, 25, 29, 30, 31, 40])

# Using broadcasting you can get a 2d array of the difference of all items
# from other items within your array
arr = time[:, None] - time
# Then find indices where the difference is less and equal to -10
x, y = np.where(arr <= -10)
# find the first occurrences of where for each item the difference is less and equal to -10 
first_acc = np.concatenate(([0], np.where(np.diff(x) != 0)[0]  + 1, [x.size]))

# use a recursive generator function to retrieve all the expected indices.
def get_ind_rec(ind=0):
    try:
        ind = y[first_acc[ind]]
        yield ind
        yield from get_ind_rec(ind)
    except: # IndexError
        pass
     
[np.sum(x) for x in np.split(signal, list(get_ind_rec()))]

[11, 9, 14, 2]

In [38]:
list(get_ind_rec())

[4, 8, 12]

In [53]:
    %%timeit
    out=[]
    prev=0
    # need to reinitialize the time array since the loop eats it
    time = np.array([0,  2,  4,  7, 10, 11, 12, 17, 21, 25, 29, 31, 40])
    while True:
        try:
            idx10 = np.where(time >=10)[0][0]
            time-=time[idx10]
            out.append(np.sum(signal[prev:idx10+1]))
            prev=idx10
        except:
            break

30.1 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [52]:
out

[12, 13, 14]

In [34]:
%%timeit
arr = time[:, None] - time
x, y = np.where(arr <= -10)
first_acc = np.concatenate(([0], np.where(np.diff(x) != 0)[0]  + 1, [x.size]))
def get_ind_rec(ind=0):
    try:
        ind = y[first_acc[ind]]
        yield ind
        yield from get_ind_rec(ind)
    except: # IndexError
        pass
     
list(get_ind_rec())

20.6 µs ± 701 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [2]:
np.array([])

SyntaxError: invalid syntax (<ipython-input-2-a1f375a7f66e>, line 1)

In [29]:
    signal = np.array([3,  2,  3,  3,  1,  3,  3,  2,  4,  3,  4,  3,  2])
    time = np.array([0,  2,  4,  7, 10, 11, 12, 17, 21, 25, 29, 31, 40])

    def count(signal, time, epoch=10):
        counts = []
        total = 0
        timestart = times[0]

        for x,t in zip(signal, time):
            total += x

            if t - timestart >= epoch:
                counts.append(total)
                total = x
                timestart = t

        return counts

    count(signal, time)

[12, 13, 14]

In [23]:
%%timeit        
count(xs, times)

5.88 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [62]:
from scipy.interpolate import interp1d

signal = np.array([3,2,3,3,1, 3, 3, 2, 4, 3, 4, 3, 2])
time = np.array([  0,2,4,7,10,11,12,17,21,25,29,30,40])

timevals = np.arange(10,time[-1]+0.0001,10,dtype=int)

npinterp = np.interp(timevals,time,signal)

interf = interp1d(time, signal, copy=False)
interf(timevals)


array([ 1. ,  3.5,  3. ,  2. ])

In [66]:
a = np.arange(1, 100)
def chunk(xs, n):
    ys = list(xs)
    random.shuffle(ys)
    size = len(ys) // n
    leftovers= ys[size*n:]
    for c, xtra in enumerate(leftovers):
        yield ys[c*size:(c+1)*size] + [ xtra ]
    for c in xrange(c+1,n):
        yield ys[c*size:(c+1)*size]

In [217]:
from itertools import chain
import numpy as np

a = np.arange(1, 100)
def chunk(xs, nlow, nhigh, shuffle=True):
    xs = np.asarray(xs)
    if shuffle:
        # shuffle, if you want
        xs = xs.copy()
        np.random.shuffle(xs)
    
    # get at least enough random chunk sizes in the specified range, ie nlow <= n <= nhigh
    ns = np.random.randint(nlow, nhigh+1, size=xs.size//nlow)
    # add up the chunk sizes to get the indices at which we'll slice up the input array
    ixs = np.add.accumulate(ns)
    # truncate ixs so that its contents are all valid indices with respect to xs
    ixs = ixs[:np.searchsorted(ixs, xs.size)]

    # yield slices from the input array
    for start,end in zip(chain([None], ixs), chain(ixs, [None])):
        yield xs[start:end]
     
list(chunk(a, 5, 10))

[array([67, 79, 17, 62, 12, 37, 70, 24]),
 array([98, 48, 88, 59, 47]),
 array([52, 60, 89, 23, 43, 44]),
 array([ 7, 27, 33, 74, 49,  2]),
 array([ 6, 51, 40, 13, 56, 45]),
 array([31,  3, 55, 10, 11, 46,  9, 42, 34]),
 array([53, 22, 95, 41, 19, 32,  4, 69, 86]),
 array([93, 68, 57, 65, 92, 76, 28, 63, 64, 58]),
 array([91, 66, 18, 99, 21]),
 array([36, 83, 15, 78,  1, 81, 97, 84]),
 array([61, 71, 25, 94, 87, 20, 85, 38]),
 array([ 8, 96, 75, 30, 77, 14, 72, 29]),
 array([35, 90, 82, 73, 39,  5, 26, 50, 16]),
 array([80, 54])]

In [216]:
nlow = 5
nhigh = 10
xs = np.arange(1, 100)

# get at least enough random chunk sizes in the specified range, ie nlow <= n <= nhigh
ns = np.random.randint(nlow, nhigh+1, size=xs.size//nlow)
# add up the chunk sizes to get the indices at which we'll slice up the input array
ixs = np.add.accumulate(ns)
# truncate ixs so that its contents are all valid indices with respect to xs
ixs[:np.searchsorted(ixs, xs.size)]
#     ixs = np.array([None] + ixs[:np.searchsorted(ixs, len(ys))].tolist() + [None])
#     return ixs
# yield slices from the input array
# for start,end in zip(chain([None], ixs), chain(ixs, [None])):
#     print(xs[start:end])

np.random.randint(nlow, nhigh+1, size=xs.size//nlow)

array([10,  9,  7,  5,  9,  6,  9,  9,  7,  6,  5,  6,  5,  5,  5,  6,  6,
        6,  9])

In [200]:
ixnp.searchsorted(ixs, xs.size)

13

In [77]:
ixs[]

[array([None, 8, 14, 19, 25, 32, 41, 50, 59, 66, 73, 82, 87, 93, None], dtype=object)]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

x = np.array([4,8,15,29,58,116,231,462,924,1848])
y = np.array([1.05,2.11,3.95,7.37,13.88,25.46,43.03,64.28,81.97,87.43])

for x1, y1 in zip(x,y):
    plt.plot(x1, y1, 'ro')

z = np.polyfit(x, y, 3)
f = np.poly1d(z)

for x1 in np.linspace(0, 1848, 110):
    plt.plot(x1, f(x1), 'b+')

plt.show()